# 🎬 PHOTOGRAPHY CONTROL — Debug & Testing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 04_PHOTOGRAPHY — CONTROL PANEL                       ║
║                Debug • Test Modules • Inspect • Preview                      ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Tester individuellement chaque composant du pipeline PHOTOGRAPHY.

---

## 📦 Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/04_PHOTOGRAPHY_WING"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print(f"✓ Drive monté")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Install dependencies
!pip install -q scipy numpy matplotlib jsonschema

## 🔍 Test 1: Vérification des Chemins

In [ ]:
from pathlib import Path

paths_to_check = [
    ("Unit Root", UNIT_ROOT),
    ("Codebase", CODEBASE),
    ("IN_SCENE", f"{UNIT_ROOT}/IN_SCENE"),
    ("OUT_CAMERA", f"{UNIT_ROOT}/OUT_CAMERA"),
]

print("=== Vérification Chemins ===")
for name, path in paths_to_check:
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")

In [ ]:
# Lister les fichiers disponibles
import os

print("\n=== Fichiers IN_SCENE ===")
in_scene_path = f"{UNIT_ROOT}/IN_SCENE"
if Path(in_scene_path).exists():
    files = [f for f in os.listdir(in_scene_path) if not f.startswith('.')]
    if files:
        for f in files:
            print(f"   - {f}")
    else:
        print("   (vide)")

print("\n=== Fichiers OUT_CAMERA ===")
out_camera_path = f"{UNIT_ROOT}/OUT_CAMERA"
if Path(out_camera_path).exists():
    files = [f for f in os.listdir(out_camera_path) if not f.startswith('.')]
    if files:
        for f in files:
            print(f"   - {f}")
    else:
        print("   (vide)")

## 📷 Test 2: Camera Styles

In [ ]:
# Test Camera Director (mode simulation sans Blender)
from camera_director import CameraDirector, CAMERA_STYLES, MOVEMENT_SPEEDS, CUT_TYPES

print("=== Camera Director ===")
print(f"\nStyles caméra disponibles: {CAMERA_STYLES}")
print(f"\nVitesses de mouvement:")
for speed, mult in MOVEMENT_SPEEDS.items():
    print(f"  - {speed}: {mult}x")

print(f"\nTypes de cuts:")
for cut_type, config in CUT_TYPES.items():
    print(f"  - {cut_type}: FOV={config['fov']}°, distance={config['distance_mult']}x")

In [ ]:
# Test création director
director = CameraDirector(verbose=True)

# Simulation de configuration
test_config = {
    "camera": {
        "style": "orbit",
        "movement": "slow"
    },
    "lighting": {
        "style": "dramatic",
        "intensity": 1.2
    }
}

print(f"\nConfig de test:")
import json
print(json.dumps(test_config, indent=2))

## 🎬 Test 3: Cuts Engine

In [ ]:
from cuts_engine import CutsEngine, CUT_PRESETS, TRANSITION_TYPES

print("=== Cuts Engine ===")
print(f"\nPresets de cuts:")
for name, preset in CUT_PRESETS.items():
    roll_info = f", roll={preset['roll']}°" if preset.get('roll', 0) != 0 else ""
    print(f"  - {name}: FOV={preset['fov']}°, dist={preset['distance_mult']}x{roll_info}")

print(f"\nTypes de transitions:")
for name, config in TRANSITION_TYPES.items():
    print(f"  - {name}: {config['blend_frames']} frames de blend")

In [ ]:
# Test génération auto-cuts
engine = CutsEngine(None, None, verbose=True)

auto_cuts = engine.generate_auto_cuts(
    frame_start=0,
    frame_end=500,
    interval=100,
    variety=True
)

print(f"\nCuts générés:")
for cut in auto_cuts:
    print(f"  Frame {cut['frame']:4d}: {cut['type']:20s} ({cut['transition']})")

## 💡 Test 4: Lighting Rig

In [ ]:
from lighting_rig import LightingRig, LIGHTING_STYLES, COLOR_TEMPS, NEON_COLORS

print("=== Lighting Rig ===")
print(f"\nStyles d'éclairage: {LIGHTING_STYLES}")

print(f"\nTempératures couleur (Kelvin):")
for temp, rgb in sorted(COLOR_TEMPS.items()):
    r, g, b = [int(v * 255) for v in rgb]
    print(f"  {temp}K: RGB({r}, {g}, {b})")

print(f"\nCouleurs néon:")
for name, rgb in NEON_COLORS.items():
    r, g, b = [int(v * 255) for v in rgb]
    print(f"  {name}: RGB({r}, {g}, {b})")

In [ ]:
# Test application styles (simulation)
rig = LightingRig(verbose=True)

print("\n--- Test des styles ---")
for style in LIGHTING_STYLES:
    print(f"\n[{style.upper()}]")
    rig.apply_style(style, intensity=1.0, color_temp=5500)
    rig.operations = []  # Reset pour le prochain test

## 🎞️ Test 5: Keyframe Animator

In [ ]:
from keyframe_animator import KeyframeAnimator, EASING_FUNCTIONS
import matplotlib.pyplot as plt
import numpy as np

print("=== Keyframe Animator ===")
print(f"\nFonctions d'easing disponibles: {list(EASING_FUNCTIONS.keys())}")

In [ ]:
# Visualisation des fonctions d'easing
t = np.linspace(0, 1, 100)

plt.figure(figsize=(14, 8))

easings_to_show = ["linear", "ease_in", "ease_out", "ease_in_out", 
                   "ease_in_cubic", "ease_out_cubic", "bounce"]

for easing in easings_to_show:
    func = EASING_FUNCTIONS[easing]
    values = [func(ti) for ti in t]
    plt.plot(t, values, label=easing, linewidth=2)

plt.xlabel('t (temps normalisé)', fontsize=12)
plt.ylabel('Valeur (0-1)', fontsize=12)
plt.title('Fonctions d\'Easing pour Animation Caméra', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Test animation path
animator = KeyframeAnimator(verbose=True)

# Définir un chemin de caméra
path = [
    (0, -10, 3),
    (5, -8, 4),
    (10, -5, 5),
    (8, 0, 4),
    (0, 5, 3)
]

print("\nAnimation path test:")
animator.animate_path(path, frame_start=0, frame_end=200, easing="ease_in_out")
print(f"Opérations: {len(animator.get_operations())}")

## 🚀 Test 6: Dry Run Pipeline Complet

In [ ]:
# Test dry-run du script principal
!python {CODEBASE}/EXO_04_PHOTOGRAPHY.py --help

In [ ]:
# Créer un PRODUCTION_PLAN.JSON de test
test_plan = {
    "project": "EXODUS_TEST",
    "scenes": [
        {
            "scene_id": 1,
            "camera": {
                "style": "orbit",
                "movement": "slow",
                "cuts": [
                    {"frame": 0, "type": "wide"},
                    {"frame": 120, "type": "medium"},
                    {"frame": 240, "type": "closeup"}
                ]
            },
            "lighting": {
                "style": "dramatic",
                "intensity": 1.2,
                "color_temp": 4500
            }
        },
        {
            "scene_id": 2,
            "camera": {
                "style": "dolly",
                "movement": "medium",
                "cuts": [
                    {"frame": 0, "type": "wide"},
                    {"frame": 180, "type": "dutch_angle"}
                ]
            },
            "lighting": {
                "style": "neon",
                "intensity": 1.0
            }
        }
    ]
}

# Sauvegarder le plan de test
test_plan_path = f"{UNIT_ROOT}/IN_SCENE/TEST_PRODUCTION_PLAN.json"
with open(test_plan_path, 'w') as f:
    json.dump(test_plan, f, indent=2)
print(f"✓ Plan de test créé: {test_plan_path}")

In [ ]:
# Dry-run avec le plan de test
# Note: nécessite un fichier .blend dans IN_SCENE pour fonctionner complètement

# !python {CODEBASE}/EXO_04_PHOTOGRAPHY.py \
#     --drive-root {DRIVE_ROOT} \
#     --production-plan TEST_PRODUCTION_PLAN.json \
#     --dry-run -v

## 📊 Inspection Rapport Photography

In [ ]:
def inspect_photography_report(report_path):
    """Inspecte un rapport de photographie."""
    with open(report_path, 'r') as f:
        report = json.load(f)
    
    print(f"=== Photography Report ===")
    print(f"Version: {report.get('version', 'N/A')}")
    print(f"Status: {report.get('status', 'N/A')}")
    print(f"Timestamp: {report.get('timestamp', 'N/A')}")
    
    summary = report.get('summary', {})
    print(f"\nRésumé:")
    print(f"  Total scènes: {summary.get('scenes_total', 0)}")
    print(f"  Traitées: {summary.get('scenes_processed', 0)}")
    print(f"  Échouées: {summary.get('scenes_failed', 0)}")
    
    print(f"\nDétail par scène:")
    for scene in report.get('scenes', []):
        print(f"\n  Scene {scene.get('scene_id')}:")
        print(f"    Status: {'✓' if scene.get('success') else '✗'}")
        print(f"    Camera: {scene.get('camera', {}).get('style', 'N/A')}")
        print(f"    Lighting: {scene.get('lighting', {}).get('style', 'N/A')}")
        print(f"    Cuts: {scene.get('camera', {}).get('cuts_count', 0)}")

# Exemple d'utilisation
# inspect_photography_report(f"{UNIT_ROOT}/OUT_CAMERA/photography_report.json")

---
## ✅ Checklist Debug

- [ ] Chemins vérifiés
- [ ] Camera styles fonctionnent
- [ ] Cuts engine génère correctement
- [ ] Lighting rigs configurés
- [ ] Keyframe animator fluide
- [ ] Dry-run passe